In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
STX_df= pd.read_csv(r'C:\Users\sagni\Desktop\ASSIGNMENT1 SNT\STX.csv')
WD_df = pd.read_csv(r'C:\Users\sagni\Desktop\ASSIGNMENT1 SNT\WD.csv')
UL_df = pd.read_csv(r'C:\Users\sagni\Desktop\ASSIGNMENT1 SNT\UL.csv')

In [ ]:
plt.plot(STX_df['Date'],STX_df['Adj Close'])

In [ ]:
plt.plot(UL_df['Date'],UL_df['Adj Close'])

In [ ]:
plt.plot(WD_df['Date'],WD_df['Adj Close'])

In [ ]:
def drop_date(df):
    df.drop(['Date'],axis=1,inplace=True)

In [ ]:
STX_df1 = drop_date(STX_df)
WD_df1 = drop_date(WD_df)
UL_df1 = drop_date(UL_df)

In [ ]:
def transform_into_x_and_y(df):
    X = df.drop(['Adj Close'],axis= 1)
    y = df['Adj Close']
    return X,y

In [ ]:
STX_X,STX_y = transform_into_x_and_y(STX_df)
WD_X,WD_y = transform_into_x_and_y(WD_df)
UL_X,UL_y = transform_into_x_and_y(UL_df)

In [ ]:
len(UL_X)

In [ ]:
def split_data(X,y,percent):
    xtrain = X[:int(X.shape[0]*percent)]
    xtest = X[int(X.shape[0]*percent):]
    ytrain = y[:int(X.shape[0]*percent)]

    ytest = y[int(X.shape[0]*percent):]
    return xtrain,xtest,ytrain,ytest

In [ ]:
STX_xtrain,STX_xtest,STX_ytrain,STX_ytest = split_data(STX_X,STX_y,0.8)

In [ ]:
UL_xtrain,UL_xtest,UL_ytrain,UL_ytest = split_data(UL_X,UL_y,0.8)

In [ ]:
WD_xtrain,WD_xtest,WD_ytrain,WD_ytest = split_data(WD_X,WD_y,0.8)

# LINEAR REGRESSION

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
def pipeline_for_linear_model(xtrain,ytrain,model):
    pipe = Pipeline(steps=[('sscaler',StandardScaler()),('model',model)])
    df_pipe = pipe.fit(xtrain,ytrain)
    return df_pipe

In [ ]:
STX_pipe_linear = pipeline_for_linear_model(STX_xtrain,STX_ytrain,LinearRegression())
WD_pipe_linear = pipeline_for_linear_model(WD_xtrain,WD_ytrain,LinearRegression())
UL_pipe_linear = pipeline_for_linear_model(UL_xtrain,UL_ytrain,LinearRegression())

# MAE,MSE,RMSE

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
def errors(df_pipe,xtest,ytest):
    ypredict = df_pipe.predict(xtest)
    print('MAE score is ',mean_absolute_error(ytest,ypredict))
    print('MSE score is ',mean_squared_error(ytest,ypredict))
    print('RMSE score is ',np.sqrt(mean_squared_error(ytest,ypredict)))
    print('R2 score is ',df_pipe.score(xtest,ytest))

In [ ]:
print('Error in model of STX')
errors(STX_pipe_linear,STX_xtest,STX_ytest)

In [ ]:
print('Error in model of WD')
errors(WD_pipe_linear,WD_xtest,WD_ytest)

In [ ]:
print('Error in model of UL')
errors(UL_pipe_linear,UL_xtest,UL_ytest)

# SVC

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

In [ ]:
STX_pipe_svr = pipeline_for_linear_model(STX_xtrain,STX_ytrain,SVR())
WD_pipe_svr = pipeline_for_linear_model(WD_xtrain,WD_ytrain,SVR())
UL_pipe_svr = pipeline_for_linear_model(UL_xtrain,UL_ytrain,SVR())

In [ ]:
errors(STX_pipe_svr,STX_xtest,STX_ytest)

In [ ]:
errors(WD_pipe_svr,WD_xtest,WD_ytest)

In [ ]:
errors(UL_pipe_svr,UL_xtest,UL_ytest)

# DECISION TREE REGRESSOR

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
param_grid1 = {'max_depth':[50,100,150,200,250],'max_leaf_nodes':[10,20,30,40,50]}

In [ ]:
def model_with_grid(pipe,xtrain,ytrain,param_grid1):
    grid = GridSearchCV(pipe,param_grid=param_grid1,cv=5)
    grid.fit(xtrain,ytrain)
    return grid

In [ ]:
STX_grid_dtr = model_with_grid(DecisionTreeRegressor(random_state=0),STX_xtrain,STX_ytrain,param_grid1)
errors(STX_grid_dtr,STX_xtest,STX_ytest)

In [ ]:
UL_grid_dtr = model_with_grid(DecisionTreeRegressor(random_state=0),UL_xtrain,UL_ytrain,param_grid1)
errors(UL_grid_dtr,UL_xtest,UL_ytest)

In [ ]:
WD_grid_dtr = model_with_grid(DecisionTreeRegressor(random_state=0),WD_xtrain,WD_ytrain,param_grid1)
errors(WD_grid_dtr,WD_xtest,WD_ytest)

# ELASTIC NET

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
grid_elasticnet = {'elastic__alpha':[0.1,0.5,1,5,10],'elastic__l1_ratio' : [0.1,0.25,0.5,0.75,0.9]}

In [ ]:
STX_elastic = model_with_grid(Pipeline(steps=[('scaler',StandardScaler()),('elastic',ElasticNet())]),STX_xtrain,STX_ytrain,grid_elasticnet)
errors(STX_elastic,STX_xtest,STX_ytest)

In [ ]:
UL_elastic = model_with_grid(Pipeline(steps=[('scaler',StandardScaler()),('elastic',ElasticNet())]),UL_xtrain,UL_ytrain,grid_elasticnet)
errors(UL_elastic,UL_xtest,UL_ytest)

In [ ]:
WD_elastic = model_with_grid(Pipeline(steps=[('scaler',StandardScaler()),('elastic',ElasticNet())]),WD_xtrain,WD_ytrain,grid_elasticnet)
errors(WD_elastic,WD_xtest,WD_ytest)

# K-NEAREST NEIGHBOUR REGRESSOR

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
grid_neighbor = {'n_neighbors':[3,5,7,10,12,15]}

In [ ]:
STX_Kneighbors = model_with_grid(KNeighborsRegressor(),STX_xtrain,STX_ytrain,grid_neighbor)
errors(STX_Kneighbors,STX_xtest,STX_ytest)

In [ ]:
UL_Kneighbors = model_with_grid(KNeighborsRegressor(),UL_xtrain,UL_ytrain,grid_neighbor)
errors(UL_Kneighbors,UL_xtest,UL_ytest)

In [ ]:
WD_Kneighbors = model_with_grid(KNeighborsRegressor(),WD_xtrain,WD_ytrain,grid_neighbor)
errors(WD_Kneighbors,WD_xtest,UL_ytest)